This code calculates plant biomass and soil carbon stocks across Y2Y. It first builds plant carbon and soil carbon layers, and then executes zonal statistics across various delineations of Y2Y. I also compared Y2Y carbon stocks to carbon stocks across Canada.

## Load packages and initialize GEE

In [1]:
# import packages
import ee
import geemap
import pandas as pd

In [2]:
# authenticate the EE api
ee.Authenticate()

True

In [3]:
# initialize the EE api
ee.Initialize(project='ee-bermane')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


## Define GEE datasets

In [4]:
# define EE datasets
# biomass
biomass = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
rsr = ee.Image("projects/ee-bermane/assets/Root_shoot_ratio_Map_Merged")

# soc
soc_olm = ee.Image("projects/ee-bermane/assets/soc_0_1m_kg_m2_olm")
soc_sothe = ee.Image(
    "projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3")

# peatlands
peat = ee.Image("projects/sat-io/open-datasets/GLOBAL-PEATLAND-DATABASE").unmask()

# landcover
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")

# netflux
netflux = ee.ImageCollection(
    "projects/wri-datalab/gfw-data-lake/net-flux-forest-extent-per-ha-v1-3-2-2001-2023/net-flux-global-forest-extent-per-ha-2001-2023"
)

# vector
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
y2y_ecoregions = ee.FeatureCollection(
    "projects/ee-bermane/assets/y2y_ecoregions")
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
rra = ee.FeatureCollection("projects/ee-bermane/assets/ross_river_ipca")
countries = ee.FeatureCollection("USDOS/LSIB/2017")
y2y_wdpa_oecm = ee.FeatureCollection("projects/ee-bermane/assets/Y2Y_pa_oecm_simplified_final_split")

## Define functions

In [5]:
# Define function to calculate zonal statistics
# specify image to use and reducer type (sum(), mean())
def calc_zonal_stats(feature, image_list, reducer_list):

    # loop through image and reducer list
    for image, reducer in zip(image_list, reducer_list):

        # calculate zonal stats across features
        stats = image.reduceRegion(
            reducer=reducer,
            geometry=feature.geometry(),
            crs=image.select(0).projection(),
            crsTransform=image.select(
                0).projection().getInfo().get('transform'),
            maxPixels=1e30
        )

        # add zonal stats to feature properties
        feature = feature.set(stats)

    return feature

# Define function to calculate feature area in km2

# def calc_feature_area(feature):
#     area_km2 = feature.geometry().area().divide(1e6)
#     return feature.set({'area_km2_gee': area_km2})

# define function to calculate additional carbon stock values
# since pixel sizes vary instead of taking the mean of all density values from the rasters
# we are calculating the total carbon and dividing it by the area in ha
# can either use the area of the extent of the non-zero raster values 
# or the area of the extent of the features

def calc_addl_stats(df):
    # calc total biomass
    df['tob_t'] = df['agb_t'] + df['bgb_t']

    # calc densities
    df['agb_t_ha'] = df['agb_t'] / df['pixel_area_ha']
    df['bgb_t_ha'] = df['bgb_t'] / df['pixel_area_ha']
    df['tob_t_ha'] = df['tob_t'] / df['pixel_area_ha']
    df['soc_t_ha'] = df['soc_t'] / df['pixel_area_ha']
    # df['agb_t_ha'] = df['agb_t'] / df['pixel_area_agb_extent_ha']
    # df['bgb_t_ha'] = df['bgb_t'] / df['pixel_area_agb_extent_ha']
    # df['tob_t_ha'] = df['tob_t'] / df['pixel_area_agb_extent_ha']
    # df['soc_t_ha'] = df['soc_t'] / df['pixel_area_soc_extent_ha']

    # calc % of storage
    df['perc_tob'] = (df['tob_t'] / df['tob_t'].sum()) * 100
    df['perc_soc'] = (df['soc_t'] / df['soc_t'].sum()) * 100

    # calc % of area
    df['perc_tob_extent'] = (
        df['pixel_area_agb_extent_ha'] / df['pixel_area_agb_extent_ha'].sum()) * 100
    df['perc_soc_extent'] = (
        df['pixel_area_soc_extent_ha'] / df['pixel_area_soc_extent_ha'].sum()) * 100
    df['perc_area'] = (df['pixel_area_ha'] / df['pixel_area_ha'].sum()) * 100
    # df['perc_tob_extent'] = (
    #     df['pixel_area_agb_extent_ha'] / df['pixel_area_agb_extent_ha'].sum()) * 100
    # df['perc_soc_extent'] = (
    #     df['pixel_area_soc_extent_ha'] / df['pixel_area_soc_extent_ha'].sum()) * 100
    # df['perc_area'] = (df['pixel_area_ha'] / df['pixel_area_ha'].sum()) * 100

    # return df
    return df

# define function to calculate zonal stats over landcover types
# currently only calculates sum in the same way as using the function above


def calc_lc_stats(image, landcover, geo, scale):
    # loop through image band names
    for index, band in enumerate(image.bandNames().getInfo()):
        # select band and add lc
        img = image.select(band).addBands(landcover)

        # take sum of band grouped by lc
        stats = img.reduceRegion(
            reducer=ee.Reducer.sum().group(groupField=1, groupName='lc'),
            geometry=geo,
            scale=scale,
            crs=img.select(0).projection(),
            # crsTransform=img.select(
            #    0).projection().getInfo().get('transform'),
            maxPixels=1e30
        )

        # save dictionary in df
        if index == 0:
            df = pd.DataFrame(stats.getInfo().get('groups')
                              ).rename(columns={'sum': band})
        else:
            df = df.merge(pd.DataFrame(stats.getInfo().get(
                'groups')).rename(columns={'sum': band}), on='lc')

    # return df
    return df

# Define function to calculate zonal statistics over different biome/lc masks
# specify mask, feature, image to use and reducer type (sum(), mean())


def calc_zonal_stats_biome_lc_mask(mask, feature, image_list, reducer_list):

    # create empty dictionary
    stats = {}

    # loop through image and reducer list
    for image, reducer in zip(image_list, reducer_list):

        # calculate zonal stats across features
        stats_append = [
            image
            .updateMask(mask)
            .reduceRegion(
                reducer=reducer,
                geometry=feature.geometry(),
                crs=image.select(0).projection(),
                crsTransform=image.select(
                    0).projection().getInfo().get('transform'),
                maxPixels=1e30
            )
        ]

        # add to dictionary
        stats.update(stats_append[0].getInfo())

    # return dictionary
    return stats

## Define Ecosystems

In [6]:
# define ecosystem masks
# Rasterize y2y_biomes to use for ecosystem classification based on BIOME_ID:
biomes_raster = y2y_biomes.reduceToImage(
    properties=['BIOME_ID'], reducer=ee.Reducer.first())

# TEMPERATE FORESTS --> FORESTRY
# create a mask based on forest lc types
# and not peatlands (1, 2)
# and biomes mask (5, 8, 13)
temperate_forest_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(peat.neq(1)
         .And(peat.neq(2)))
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8))
         .Or(biomes_raster.eq(13)))
)

# NORTHERN FORESTS --> FORESTRY
# create a mask based on forest lc types
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_forest_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(peat.neq(1)
         .And(peat.neq(2)))
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
# create a mask based on grasslands and shrublands LC type (8, 10)
# and not peatlands (1, 2)
# and biomes mask (8)
temperate_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10)))
    .And(peat.neq(1)
         .And(peat.neq(2)))
    .And(biomes_raster.eq(8))
)

# MONTANE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
# create a mask based on grasslands and shrublands LC type (8, 10)
# and not peatlands (1, 2)
# and biomes mask (5)
montane_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10)))
    .And(peat.neq(1)
         .And(peat.neq(2)))
    .And(biomes_raster.eq(5))
)

# NORTHERN GRASSLANDS AND SHRUBLANDS --> MINING/OIL/GAS/DEV
# create a mask based on grasslands, shrublands, lichen-moss LC type (8, 10, 11, 12, 13)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10))
     .Or(lc.eq(11))
     .Or(lc.eq(12))
     .Or(lc.eq(13)))
    .And(peat.neq(1)
         .And(peat.neq(2)))
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE WETLANDS --> AGRICULTURE
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_wetlands_mask = (
    lc.eq(14)
    .And(peat.neq(1)
         .And(peat.neq(2)))
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN WETLANDS --> MINING/OIL/GAS/DEV
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_wetlands_mask = (
    lc.eq(14)
    .And(peat.neq(1)
         .And(peat.neq(2)))
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE PEATLANDS --> AGRICULTURE
# create a mask based on peatlands LC type (1, 2)
# and not urban or cropland (15, 17)
# and biomes mask (5, 8)
# and where there is forest (1-6) and peat (1, 2) in the desert biome (13)
temperate_peatlands_mask = (
    ((peat.eq(1)
     .Or(peat.eq(2)))
    .And(lc.neq(15)
         .And(lc.neq(17))
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))))
    .Or(lc.gte(1)
        .And(lc.lte(6))
        .And((peat.eq(1)
              .Or(peat.eq(2))))
        .And(biomes_raster.eq(13)))
)

# NORTHERN PEATLANDS --> MINING/OIL/GAS/DEV
# create a mask based on peatlands LC type (1, 2)
# and not urban or cropland (15, 17)
# and biomes mask (6, 11)
northern_peatlands_mask = (
    (peat.eq(1)
     .Or(peat.eq(2)))
    .And(lc.neq(15)
         .And(lc.neq(17)))
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE BARREN LANDS --> MINING/OIL/GAS/DEV
# LC type (16)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_barren_mask = (
    lc.eq(16)
    .And(peat.neq(1)
         .And(peat.neq(2)))
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN BARREN LANDS --> MINING/OIL/GAS/DEV
# create a mask based on barren lands LC type (16)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_barren_mask = (
    lc.eq(16)
    .And(peat.neq(1)
         .And(peat.neq(2)))
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# DESERT --> NOT MANAGEABLE
# biomes mask (13)
# and not forest (1-6)
# and not urban or cropland (15, 17)
desert_mask = (
    lc.gte(7)
    .And(lc.neq(15)
         .And(lc.neq(17)))
    .And(biomes_raster.eq(13))
)

# URBAN AND BUILT-UP/CROPLAND --> NOT MANAGEABLE
# LC type (15, 17)
# all biomes
urban_crop_mask = (
    (lc.eq(15)
     .Or(lc.eq(17)))
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(6))
         .Or(biomes_raster.eq(8))
         .Or(biomes_raster.eq(11))
         .Or(biomes_raster.eq(13)))
)

# create ecosystem mask
eco_mask = (
    temperate_forest_mask
    .Or(northern_forest_mask)
    .Or(temperate_grass_shrub_mask)
    .Or(montane_grass_shrub_mask)
    .Or(northern_grass_shrub_mask)
    .Or(temperate_wetlands_mask)
    .Or(northern_wetlands_mask)
    .Or(temperate_peatlands_mask)
    .Or(northern_peatlands_mask)
    .Or(temperate_barren_mask)
    .Or(northern_barren_mask)
    .Or(desert_mask)
    .Or(urban_crop_mask)
)

# Vegetation cover for biomass replaceability in peatlands
# TEMPERATE FORESTS --> FORESTRY
# create a mask based on forest lc types
# and biomes mask (5, 8, 13)
temperate_forest_bio_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8))
         .Or(biomes_raster.eq(13)))
)

# NORTHERN FORESTS --> FORESTRY
# create a mask based on forest lc types
# and biomes mask (6, 11)
northern_forest_bio_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# not masking out protected areas
# # create a mask out of wdpa layer
# # to remove already protected areas from manageable carbon
# wdpa_mask = ee.Image.constant(1).clip(y2y_wdpa).mask()


## Calculate biomass stock from ESA CCI data

In [7]:
# calc 2022 ESA CCI biomass
agb = biomass.filter(ee.Filter.stringContains("system:index", "2022")).first()

# create image for litter using Harris ratio (4%)
# leave out dead wood since it is likely sensed already
# mask litter only for forested LC types
# 1: "#033e00",  # Temperate or sub-polar needleleaf forest
# 2: "#939b71",  # Sub-polar taiga needleleaf forest
# 3: "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
# 4: "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
# 5: "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
# 6: "#6b7d2c",  # Mixed forest

litter_mask = lc.gte(1).And(lc.lte(6))

litter = agb.select(['AGB']).multiply(0.04).multiply(litter_mask)

# create image for BGB (using global rsr map)
bgb = agb.select(['AGB']).multiply(rsr).rename('BGB')

# add AGB + litter and BGB together
bio = agb.select(['AGB']).add(litter).addBands(
    bgb).rename(['agb_t_ha', 'bgb_t_ha'])

# multiply values by 0.47 to get carbon density
# 0.47 used by Harris et al. (2021)
# mask water (18) and snow/ice (19)
bio = bio.multiply(0.47).updateMask(
    lc.neq(18).And(lc.neq(19)))

# compute per-pixel area in ha
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# Create carbon layer mask to filter pixel area raster
carbon_mask = bio.select(['agb_t_ha']).neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_agb_extent = pixel_area_ha.updateMask(carbon_mask)

# calculate total biomass c per pixel and rename bands
bio_stock = bio.multiply(
    pixel_area_ha
).rename(
    ['agb_t', 'bgb_t']
).addBands(
    pixel_area_agb_extent.rename(['pixel_area_agb_extent_ha'])
).addBands(
    pixel_area_ha.rename(['pixel_area_ha'])
)

## Calculate soil carbon from Sothe Canada data and Open Land Map US data

In [8]:
# calc open land map soc
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_olm = soc_olm.multiply(10).rename(
    'soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# reproject to match sothe to blend images
soc_olm_reproj_sothe = soc_olm.resample('bilinear').toFloat()

In [9]:
# calc sothe soc
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_sothe = soc_sothe.multiply(10).rename(
    'soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# blend sothe and olm carbon across y2y
soc_blend = ee.ImageCollection([soc_olm_reproj_sothe, soc_sothe]).mosaic().rename('soc_t_ha').reproject(
    crs=soc_sothe.projection(),
    crsTransform=soc_sothe.projection().getInfo().get('transform')
).updateMask(lc.neq(18).And(lc.neq(19)))  # mask snow/ice, water

# Create soc layer mask to filter pixel area raster
soc_mask = soc_blend.mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_soc_extent = pixel_area_ha.updateMask(soc_mask)

# multiply by pixel area to get total carbon per pixel
soc_blend_stock = soc_blend.multiply(pixel_area_ha).rename('soc_t').addBands(
    pixel_area_soc_extent.rename(['pixel_area_soc_extent_ha'])
).addBands(
    pixel_area_ha.rename(['pixel_area_ha'])
)

## Check scale of output layers

In [10]:
# check scale of layers
print('Biomass Carbon Density Scale: ',
      bio.projection().nominalScale().getInfo())
print('Biomass Carbon Total Scale: ',
      bio_stock.select(['agb_t']).projection().nominalScale().getInfo())
# print('SOC Density Sothe Scale: ', soc_sothe.projection().nominalScale().getInfo())
print('SOC Density Blend Scale: ', soc_blend.select(
    ['soc_t_ha']).projection().nominalScale().getInfo())
print('SOC Total Blend Scale: ',
      soc_blend_stock.select(['soc_t']).projection().nominalScale().getInfo())
# print('SOC Density OLM Scale: ', soc_olm.projection().nominalScale().getInfo())

Biomass Carbon Density Scale:  98.95065848290984
Biomass Carbon Total Scale:  98.95065848290984
SOC Density Blend Scale:  250.00021298953834
SOC Total Blend Scale:  250.00021298953834


## Compute zonal statistics and convert results to dataframe

In [11]:
# define image and reducer list for zonal stats
img_list = [bio_stock, soc_blend_stock]
redu_list = [ee.Reducer.sum(), ee.Reducer.sum()]

# define separate

# define output folder for exports
out_folder = './outputs/'

In [12]:
# Compute zonal statistics across Y2Y region
y2y_carbon = y2y.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
y2y_df = ee.data.computeFeatures({
    'expression': y2y_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# add density values
y2y_df = calc_addl_stats(y2y_df)

# drop geometry, round values and export as csv
y2y_df.drop('geo', axis=1).round(2).to_csv(
    out_folder + 'y2y_carbon_stocks.csv', index=False)

In [13]:
# compute zonal statistics across ecosystems
# create dictionary with masks
mask_dict = {
    'Temperate Forest':temperate_forest_mask, 
    'Northern Forest':northern_forest_mask, 
    'Temperate Grass Shrub':temperate_grass_shrub_mask,
    'Montane Grass Shrub':montane_grass_shrub_mask, 
    'Northern Grass Shrub':northern_grass_shrub_mask, 
    'Temperate Wetlands':temperate_wetlands_mask,
    'Northern Wetlands':northern_wetlands_mask, 
    'Temperate Peatlands':temperate_peatlands_mask, 
    'Northern Peatlands':northern_peatlands_mask,
    'Temperate Barren':temperate_barren_mask, 
    'Northern Barren':northern_barren_mask,
    'Desert':desert_mask,
    'Urban, Built-up, Cropland':urban_crop_mask
    }

# create empty dataframe
biome_lc_df = pd.DataFrame()

# loop through biome/lc masks
for name, mask in mask_dict.items():
    df_hold = calc_zonal_stats_biome_lc_mask(mask, y2y, img_list, redu_list)
    biome_lc_df = pd.concat([biome_lc_df, pd.DataFrame(df_hold, index=[name])])

# drop duplicate pixel_area_ha_column
# lc_df = lc_df.drop(columns=['pixel_area_ha_y']).rename(
#     columns={'pixel_area_ha_x': 'pixel_area_ha'})

# calc additional stats
biome_lc_df = calc_addl_stats(biome_lc_df)

# round values and export as csv
biome_lc_df.round(2).to_csv(
    out_folder + 'y2y_ecosystem_masks_carbon_stocks.csv', index=True)

In [14]:
# # get canada and y2y geometry
# canada = countries.filter(ee.Filter.eq('COUNTRY_NA', 'Canada'))
# can_geo = canada.geometry()
# y2y_geo = y2y.geometry()

# # get intersection of geometries
# y2y_can_geo = y2y_geo.intersection(can_geo)

# # create feature collection
# y2y_can = ee.FeatureCollection(y2y_can_geo)

# # Compute zonal statistics across y2y in canada
# y2y_can_carbon = y2y_can.map(lambda feature: calc_zonal_stats(
#     feature, img_list, redu_list))

# # convert to df
# y2y_can_df = ee.data.computeFeatures({
#     'expression': y2y_can_carbon,
#     'fileFormat': 'PANDAS_DATAFRAME'
# })

# # add density values
# y2y_can_df = calc_addl_stats(y2y_can_df)

# # drop geometry, round values and export as csv
# y2y_can_df.drop('geo', axis=1).round(2).to_csv(
#     out_folder + 'y2y_canada_carbon_stocks.csv', index=False)

In [15]:
# Compute zonal statistics across biomes
biome_carbon = y2y_biomes.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
biome_df = ee.data.computeFeatures({
    'expression': biome_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# add density values
biome_df = calc_addl_stats(biome_df)

# drop geometry, round values and export as csv
biome_df.drop('geo', axis=1).round(2).to_csv(
    out_folder + 'y2y_biome_carbon_stocks.csv', index=False)

In [16]:
# Compute zonal statistics across ecoregions
eco_carbon = y2y_ecoregions.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
eco_df = ee.data.computeFeatures({
    'expression': eco_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# add density values
eco_df = calc_addl_stats(eco_df)

# drop geometry, round values and export as csv
eco_df.drop('geo', axis=1).round(2).to_csv(
    out_folder + 'y2y_ecoregion_carbon_stocks.csv', index=False)

In [14]:
# Compute zonal statistics across protected areas
# combine feature collections
y2y_protected_areas = y2y_wdpa_oecm

# clip zonal stat images to collection
# since protected areas is too complex
bio_stock_prot = bio_stock.clipToCollection(y2y_protected_areas)
soc_blend_stock_prot = soc_blend_stock.clipToCollection(y2y_protected_areas)

# define image and reducer list for zonal stats
img_list_prot = [bio_stock_prot, soc_blend_stock_prot]
redu_list_prot = [ee.Reducer.sum(), ee.Reducer.sum()]

protected_areas_carbon = y2y.map(lambda feature: calc_zonal_stats(
    feature, img_list_prot, redu_list_prot))

# convert to df
protected_areas_df = ee.data.computeFeatures({
    'expression': protected_areas_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# group geometries by governance type
protected_areas_df = protected_areas_df.drop(
    ['geo'], axis=1)

# create row for non protected areas
non_protected_row = pd.DataFrame([{'agb_t': (y2y_df['agb_t'] - protected_areas_df['agb_t'].sum()).values[0],
                                   'bgb_t': (y2y_df['bgb_t'] - protected_areas_df['bgb_t'].sum()).values[0],
                                   'pixel_area_agb_extent_ha': (y2y_df['pixel_area_agb_extent_ha'] - protected_areas_df['pixel_area_agb_extent_ha'].sum()).values[0],
                                   'pixel_area_ha': (y2y_df['pixel_area_ha'] - protected_areas_df['pixel_area_ha'].sum()).values[0],
                                   'pixel_area_soc_extent_ha': (y2y_df['pixel_area_soc_extent_ha'] - protected_areas_df['pixel_area_soc_extent_ha'].sum()).values[0],
                                   'soc_t': (y2y_df['soc_t'] - protected_areas_df['soc_t'].sum()).values[0]}])

# add non protected row to df
protected_areas_df = pd.concat(
    [protected_areas_df, non_protected_row], ignore_index=True)

# add density values
protected_areas_df = calc_addl_stats(protected_areas_df)

# round values and export as csv
protected_areas_df.round(2).to_csv(
    out_folder + 'y2y_protected_areas_carbon_stocks.csv', index=False)

In [18]:
# Compute zonal statistics in Ross River IPCA
rra_carbon = rra.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
rra_df = ee.data.computeFeatures({
    'expression': rra_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# add density values
rra_df = calc_addl_stats(rra_df)

# drop geometry, round values and export as csv
rra_df.drop('geo', axis=1).round(2).to_csv(
    out_folder + 'y2y_rra_carbon_stocks.csv', index=False)

In [19]:
# Compute zonal statistics across Canada
canada = countries.filter(ee.Filter.eq('COUNTRY_NA', 'Canada'))

# clip zonal stat images to collection
# since canada is big and complex image collection
bio_stock_can = bio_stock.clipToCollection(canada)
soc_blend_stock_can = soc_blend_stock.clipToCollection(canada)

# define geometry bigger than canada
big_geo = ee.Geometry.Rectangle(
    coords=[-148, 38, -45, 85],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection,
    geodesic=False)

# Create a Feature with this geometry
big_feature = ee.Feature(big_geo, {"name": "Bigger than Canada"})

# Create a FeatureCollection
big_fc = ee.FeatureCollection([big_feature])

# define image and reducer list for zonal stats
img_list_can = [bio_stock_can, soc_blend_stock_can]
redu_list_can = [ee.Reducer.sum(), ee.Reducer.sum()]

# Compute zonal statistics across canada
canada_carbon = big_fc.map(lambda feature: calc_zonal_stats(
    feature, img_list_can, redu_list_can))

# # convert to df
# canada_df = ee.data.computeFeatures({
#     'expression': canada_carbon,
#     'fileFormat': 'PANDAS_DATAFRAME'
# })

# export to drive
# too big a process to load directly
ee.batch.Export.table.toDrive(
    collection=canada_carbon,
    description="canada_carbon_stock",
    folder="carbon_stock_outputs",
    fileFormat="CSV"
).start()

In [21]:
# save to disk from drive and reload as df
canada_df = pd.read_csv('./outputs/drive/canada_carbon_stock.csv')

# add density values
canada_df = calc_addl_stats(canada_df)

# drop geometry, round values and export as csv
canada_df.drop(['.geo', 'name', 'system:index'], axis=1).round(2).to_csv(
    out_folder + 'canada_carbon_stocks.csv', index=False)

In [ ]:
# # compute zonal statistics across landcover types
# # bio first
# lc_df = calc_lc_stats(bio_stock, lc, y2y.geometry(),
#                       bio_stock.select(0).projection().nominalScale())

# # merge with soc
# lc_df = lc_df.merge(
#     calc_lc_stats(soc_blend_stock, lc, y2y.geometry(),
#                   bio_stock.select(0).projection().nominalScale()),
#     on=['lc']
# )

# # drop duplicate pixel_area_ha_column
# lc_df = lc_df.drop(columns=['pixel_area_ha_y']).rename(
#     columns={'pixel_area_ha_x': 'pixel_area_ha'})

# # add density values
# lc_df = calc_addl_stats(lc_df)

# # set landcover mapping key
# landcover_mapping = {
#     1: "Temperate or sub-polar needleleaf forest",
#     2: "Sub-polar taiga needleleaf forest",
#     3: "Tropical or sub-tropical broadleaf evergreen forest",
#     4: "Tropical or sub-tropical broadleaf deciduous forest",
#     5: "Temperate or sub-polar broadleaf deciduous forest",
#     6: "Mixed forest",
#     7: "Tropical or sub-tropical shrubland",
#     8: "Temperate or sub-polar shrubland",
#     9: "Tropical or sub-tropical grassland",
#     10: "Temperate or sub-polar grassland",
#     11: "Sub-polar or polar shrubland-lichen-moss",
#     12: "Sub-polar or polar grassland-lichen-moss",
#     13: "Sub-polar or polar barren-lichen-moss",
#     14: "Wetland",
#     15: "Cropland",
#     16: "Barren land",
#     17: "Urban and built-up",
#     18: "Water",
#     19: "Snow and ice"
# }

# # map landcover names
# lc_df['lc'] = lc_df['lc'].map(landcover_mapping)
# lc_df

# # round values and export as csv
# lc_df.round(2).to_csv(
#     out_folder + 'y2y_lc_carbon_stocks.csv', index=False)

## Export Rasters

In [20]:
# export rasters to disk in native resolution
data_folder = './carbon_stock_data/output_layers/'

bio_out = bio.addBands(bio.select('agb_t_ha').add(
    bio.select('bgb_t_ha')).rename('tob_t_ha'))

geemap.download_ee_image(bio_out, # mask for ecosystems
                         filename=data_folder + 'biomass_2022_t_ha.tif',
                         region=y2y.geometry(),
                         crs=bio.projection().getInfo().get('crs'),
                         crs_transform=bio.projection().getInfo().get('transform'))

geemap.download_ee_image(soc_blend, # mask for ecosystems
                         filename=data_folder + 'soc_0_1m_t_ha.tif',
                         region=y2y.geometry(),
                         crs=soc_blend.projection().getInfo().get('crs'),
                         crs_transform=soc_blend.projection().getInfo().get('transform'))

Consider adjusting `region`, `scale` and/or `dtype` to reduce the biomass_2022_t_ha.tif download size (raw: 25.43 GB).


biomass_2022_t_ha.tif: |          | 0.00/25.4G (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


soc_0_1m_t_ha.tif: |          | 0.00/664M (raw) [  0.0%] in 00:00 (eta:     ?)

## Add layers to map

In [14]:
# set biomass palette
biomass_palette = [
    "#C6ECAE", "#A1D490", "#7CB970", "#57A751", "#348E32",
    "#267A29", "#176520", "#0C4E15", "#07320D", "#031807"
]

# set land cover palette
lc_palette = [
    "#033e00",  # Temperate or sub-polar needleleaf forest
    "#939b71",  # Sub-polar taiga needleleaf forest
    "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
    "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
    "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
    "#6b7d2c",  # Mixed forest
    "#b29d29",  # Tropical or sub-tropical shrubland
    "#b48833",  # Temperate or sub-polar shrubland
    "#e9da5d",  # Tropical or sub-tropical grassland
    "#e0cd88",  # Temperate or sub-polar grassland
    "#a07451",  # Sub-polar or polar shrubland-lichen-moss
    "#bad292",  # Sub-polar or polar grassland-lichen-moss
    "#3f8970",  # Sub-polar or polar barren-lichen-moss
    "#6ca289",  # Wetland
    "#e6ad6a",  # Cropland
    "#a9abae",  # Barren land
    "#db2126",  # Urban and built-up
    "#4c73a1",  # Water
    "#fff7fe",  # Snow and ice
]

In [19]:
# create a map
m = geemap.Map()

# add layers
# m.addLayer(soc_blend, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC Density Blend 0-1m")
# m.addLayer(rsr, {}, 'Root-to-Shoot Ratio Global')
# m.addLayer(bio.select(['agb_t_ha']).clip(y2y), {
#           "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB Density')
# m.addLayer(bio.select(['dpm_t_ha']).clip(y2y), {"min": 1, "max": 60, "palette": biomass_palette}, 'DPM Density')
# m.addLayer(pixel_area_agb_extent.clip(y2y), {}, 'Carbon Mask')
m.addLayer(lc.clip(y2y), {"min": 1, "max": 19,
          "palette": lc_palette}, 'Landcover')
# m.addLayer(y2y_can, {}, 'Y2Y Canada Extent')
# m.addLayer(y2y_oecm, {}, 'Y2Y OECM')
# m.addLayer(big_fc, {}, 'Big Geo')
# m.addLayer(eco_mask, {}, 'Ecosystem Mask')
m.addLayer(peat, {"min": 0, "max": 3,
          "palette": lc_palette}, "Peat Distribution")
m.addLayer(desert_mask, {}, 'Desert Mask')

# Display the map
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…